# Face Detection

## 1. Setup and Get Data

### 1.1 Install dependences

In [3]:
# ! pip install labelme tensorflow opencv-python matplotlib albumentations

In [4]:
# ! python.exe -m pip install --upgrade pip

### 1.2 Collecting imagec using OpenCV

In [5]:
import os, cv2
from uuid import uuid1
from time import sleep
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

In [6]:
image_path = os.path.join('data', 'images')
number_images = 30

In [8]:
cap = cv2.VideoCapture(0)

for image_num in range(number_images):
    print(f"img num: {image_num}")
    ret, frame = cap.read()
    image_name = os.path.join(image_path, f"{uuid1().hex}.jpg")

    cv2.imwrite(image_name, frame)
    cv2.imshow('frame', frame)
    sleep(0.5)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

img num: 0
img num: 1
img num: 2
img num: 3
img num: 4
img num: 5
img num: 6
img num: 7
img num: 8
img num: 9
img num: 10
img num: 11
img num: 12
img num: 13
img num: 14
img num: 15
img num: 16
img num: 17
img num: 18
img num: 19
img num: 20
img num: 21
img num: 22
img num: 23
img num: 24
img num: 25
img num: 26
img num: 27
img num: 28
img num: 29


### 1.3 Annotating images using labelme

In [9]:
! labelme

2025-02-27 13:04:27.746 | INFO     | labelme.config:get_config:66 - Loading config file from: C:\Users\Shermukhammad\.labelmerc


## 2. Review Datset and build image loading function

### 2.1 Import Tensorflow and dependecies

In [15]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import json

### 2.3 Limit GPU memory grow

In [24]:
gpus = tf.config.experimental.list_physical_devices('GPU')
gpus

[]

In [25]:
tf.test.is_built_with_cuda()

False